## B. Presidential results (D%, R%, two-party %)

Difficulty: Easy

Sources: MIT Election Data & Science Lab (county & state returns), state SOS, or Dave Leip’s Atlas (paid). MIT Election Lab is recommended for reproducibility.

MIT Election Lab
Quality issues: late-certified adjustments are rare but check state certification dates. Third-party vote handling—compute two-party share.
Estimated time: hours to 1 day using MIT Election Lab CSVs.

https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/42MVDX